In [1]:
import pandas as pd
from to_db import Connector

### The following cell pulls the temperature data from the database and group by date 

In [2]:
conn = Connector()
df = conn.get_data()
df["date"] = pd.to_datetime(df['date'])
data = df.groupby("date").mean()

In [3]:
from datetime import datetime

In [4]:
from time import localtime,strftime,time

In [17]:
from mariadb import Date

In [26]:
def get_daily_temp(date):
    d = Date(date.year,date.month,date.day)
    return conn.dates(d)

In [27]:
td = datetime.now()

In [28]:
daily = get_daily_temp(td.date())

In [7]:
import serial

In [8]:
from numpy import median

In [9]:
def date_range():
    obj = dcc.DatePickerSingle(
        id='date_picker',
        min_date_allowed = data.index[0],
        max_date_allowed = date.today(),
        initial_visible_month = date.today(),
        date = date.today()
    )
    
    return obj

In [10]:
def statistics():
    obj = dcc.RadioItems(
        id = "stats",
        options=[
            {'label': 'max','value':'max'},
            {'label': 'min','value':'min'},
            {'label': 'mean','value':'mean'}],
        value = 'mean',
        style = {'color':colours['radio_button'],
                'fontSize':text_size['radio_button']}
    )
    
    return obj

In [11]:
def temperature_dial():
    temp  = read_temp(ser)
    dial = go.Indicator(mode="gauge+number",value=temp,
                        title={'text':'Temperature/C'},
                        gauge = {'bar':{'color':'black'},
                                 'axis':{'range':(None, 60)},
                                 'steps':[{'range':[0,16],'color':'lightblue'},
                                          {'range':[16,28],'color':'lightgreen'},
                                          {'range':[28,60],'color':'#FF7F7F'}]
                            
                        })
    
    return dial

In [12]:
ser = serial.Serial('/dev/ttyACM0')
ser.flushInput()

In [13]:
READINGS = 20

In [14]:
def read_temp(ser):
    while True:
        readings = []
        try:
            for i in range(READINGS):
                bytes = ser.readline()
                decoded = bytes[0:len(bytes)-2].decode("utf-8")
                decoded = float(decoded)
                readings.append(decoded)
            break
        except:
            pass

    return median(decoded)

In [44]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date



stats = {"mean": df.groupby("date").mean(),"max":df.groupby("date").max(),"min":df.groupby("date").min()}


app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#777777','radio_button':'#DDDDDD'}
text_size = {'H1':48,'H2':40,'text':36,'radio_button':28}

# fig = make_subplots(rows = 1, cols= 2)

# fig.add_trace(go.Scatter(x = data.index, y = data["temperature"]),row=1,col=1)
# fig.add_trace(go.Scatter(x= today["time"], y= today["temperature"]),row=1,col=2)

fig = px.scatter(x = data.index, y = data["temperature"])
# fig = px.scatter(df, x = "date" , y = "temperature")

fig.update_yaxes(rangemode = "tozero")

fig2 = go.Figure(temperature_dial())

fig3 = px.scatter(daily,"time", "temperature",title= "Daily breakdown for: "+td.strftime("%a %d %b %Y"))
fig3.update_yaxes(rangemode = "tozero")

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'calibri'}, children=[

    html.H1(children='Room Temperature',
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),

    dcc.Graph(
        id='graph2',
        figure=fig2
    ),
    
    html.Div([html.Label("Stats: ", style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['text']}),
                statistics()
             ],style = {'textAlign': 'center',"flex":1}
    ),
    
    

    dcc.Graph(
        id='graph',
        figure=fig
    ),
    
    html.Div([html.Label("Choose date for daily breakdown: ", style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['text']}),
              html.Br(),
              date_range()
             ],style = {'textAlign': 'center',"flex":1}
    ),

    
    dcc.Graph(
        id='graph3',
        figure=fig3
    )
])

@app.callback(
    Output('graph2','figure'),
    Output('graph','figure'),
    Output('graph3','figure'),
    Input('stats','value'),
    Input('date_picker','date'))
def update_figure(stat,day):
    temp  = read_temp(ser)
    data = stats[stat]
    day = datetime.strptime(day,"%Y-%m-%d").date()
    
    daily =  get_daily_temp(day)
    

#     fig = px.scatter(df, x = "date" , y = "temperature")

    fig = px.scatter(x = data.index, y = data["temperature"],title=stat.capitalize() + " temperature over time")
    fig.update_yaxes(rangemode = "tozero")
    fig2 = go.Figure(temperature_dial())

    fig3 = px.scatter(daily,"time", "temperature",title="Temperature breakdown for: "+day.strftime("%a %d %b %Y"))
    fig3.update_yaxes(rangemode = "tozero")
#     fig.data =[]

#     fig.add_trace(go.Scatter(x = data.index, y = data["temperature"]),row=1,col=1)
#     fig.add_trace(go.Scatter(x=today["time"], y=today["temperature"]),row=1,col=2)
#     fig.update_layout(showlegend=False)
    
    return fig2,fig,fig3

In [45]:
app.run_server(mode = "external")

Dash app running on http://127.0.0.1:8050/


/home/blunt/.local/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

